# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [ ]:
# Import your libraries:

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Challenge 1 - Import and Describe the Dataset

In this lab, we will use a dataset containing information about customer preferences. We will look at how much each customer spends in a year on each subcategory in the grocery store and try to find similarities using clustering.

The origin of the dataset is [here](https://archive.ics.uci.edu/ml/datasets/wholesale+customers).

In [ ]:
# loading the data:
customers = pd.read_csv('../Wholesale customers data.csv')

#### Explore the dataset with mathematical and visualization techniques. What do you find?

Checklist:

* What does each column mean?
* Any categorical data to convert?
* Any missing data to remove?
* Column collinearity - any high correlations?
* Descriptive statistics - any outliers to remove?
* Column-wise data distribution - is the distribution skewed?
* Etc.

Additional info: Over a century ago, an Italian economist named Vilfredo Pareto discovered that roughly 20% of the customers account for 80% of the typical retail sales. This is called the [Pareto principle](https://en.wikipedia.org/wiki/Pareto_principle). Check if this dataset displays this characteristic.

In [ ]:
# Your code here:
customers.shape

In [ ]:
customers.dtypes

In [ ]:
customers.isnull().sum()

In [ ]:
customers.describe()

In [ ]:
ax = sns.heatmap(customers.corr(), linewidths=.5)

In [ ]:
# Your comment here

In [ ]:
In the heat map we appreciate that in the Region field there is no great correlation with any of the other fields, and it also remains constant.

# Challenge 2 - Data Cleaning and Transformation

If your conclusion from the previous challenge is the data need cleaning/transformation, do it in the cells below. However, if your conclusion is the data need not be cleaned or transformed, feel free to skip this challenge. But if you do choose the latter, please provide rationale.

In [ ]:
# Your code here
customers.drop(columns=['Region'], axis=1,inplace=True)
customers.head()

# Challenge 3 - Data Preprocessing

One problem with the dataset is the value ranges are remarkably different across various categories (e.g. `Fresh` and `Grocery` compared to `Detergents_Paper` and `Delicassen`). If you made this observation in the first challenge, you've done a great job! This means you not only completed the bonus questions in the previous Supervised Learning lab but also researched deep into [*feature scaling*](https://en.wikipedia.org/wiki/Feature_scaling). Keep on the good work!

Diverse value ranges in different features could cause issues in our clustering. The way to reduce the problem is through feature scaling. We'll use this technique again with this dataset.

#### We will use the `StandardScaler` from `sklearn.preprocessing` and scale our data. Read more about `StandardScaler` [here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).

*After scaling your data, assign the transformed data to a new variable `customers_scale`.*

In [ ]:
# Your import here:

from sklearn.preprocessing import StandardScaler

# Your code here:

scaler = StandardScaler()
scaler.fit(customers)
scaler.mean_
customers_scale = scaler.transform(customers)
customers_scale1= pd.DataFrame(customers_scale)
customers_scale1.columns= ['Channel', 'Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen']
customers_scale1.head()

# Challenge 3 - Data Clustering with K-Means

Now let's cluster the data with K-Means first. Initiate the K-Means model, then fit your scaled data. In the data returned from the `.fit` method, there is an attribute called `labels_` which is the cluster number assigned to each data record. What you can do is to assign these labels back to `customers` in a new column called `customers['labels']`. Then you'll see the cluster results of the original data.

In [ ]:
# Your code here:
from sklearn.cluster import KMeans

kmeans = [KMeans(n_clusters=i, init='random',
            n_init=10, max_iter=300,
            tol=1e-04, random_state=42) 
            for i in range(1, 22)]
score = [kmeans[i].fit(customers_scale1).score(customers_scale1) for i in range(len(kmeans))]
plt.plot(range(1, 22),score)
plt.xlabel('Número de clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

Count the values in `labels`.

In [ ]:
# Your code here:
kmeans = KMeans(n_clusters=14, random_state=42).fit(customers_scale1)

customers['labels']= kmeans.labels_
customers['labels'].value_counts()

# Challenge 4 - Data Clustering with DBSCAN

Now let's cluster the data using DBSCAN. Use `DBSCAN(eps=0.5)` to initiate the model, then fit your scaled data. In the data returned from the `.fit` method, assign the `labels_` back to `customers['labels_DBSCAN']`. Now your original data have two labels, one from K-Means and the other from DBSCAN.

In [ ]:
# Your code here
from sklearn.cluster import DBSCAN

clustering = DBSCAN(eps=0.5, min_samples=4).fit(customers_scale1)
customers['labels_DBSCAN'] = clustering.labels_


Count the values in `labels_DBSCAN`.

In [ ]:
# Your code here
customers['labels_DBSCAN'].value_counts()

# Challenge 5 - Compare K-Means with DBSCAN

Now we want to visually compare how K-Means and DBSCAN have clustered our data. We will create scatter plots for several columns. For each of the following column pairs, plot a scatter plot using `labels` and another using `labels_DBSCAN`. Put them side by side to compare. Which clustering algorithm makes better sense?

Columns to visualize:

* `Detergents_Paper` as X and `Milk` as y
* `Grocery` as X and `Fresh` as y
* `Frozen` as X and `Delicassen` as y

Visualize `Detergents_Paper` as X and `Milk` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:
def get_scatter_plot(x_, y_, col):
    plt.scatter(customers[x_], customers[y_], c=customers[col], alpha=0.5)
    plt.xlabel(x_)
    plt.ylabel(y_)
    plt.title('Visualization {} vs {} by {}'.format(x_, y_, col))
    plt.show()

In [ ]:
get_scatter_plot('Detergents_Paper','Milk', 'labels')
get_scatter_plot('Detergents_Paper','Milk', 'labels_DBSCAN')

Visualize `Grocery` as X and `Fresh` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:
get_scatter_plot('Grocery','Fresh', 'labels')
get_scatter_plot('Grocery','Fresh', 'labels_DBSCAN')


Visualize `Frozen` as X and `Delicassen` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:
get_scatter_plot('Frozen','Delicassen', 'labels')
get_scatter_plot('Frozen','Delicassen', 'labels_DBSCAN')


Let's use a groupby to see how the mean differs between the groups. Group `customers` by `labels` and `labels_DBSCAN` respectively and compute the means for all columns.

In [ ]:
# Your code here:
customers_labels = customers.groupby('labels').mean()
customers_labels


In [ ]:
ax = sns.heatmap(customers_labels.corr(), linewidths=.5)

In [ ]:
customers_labels_DBSCAN = customers.groupby('labels_DBSCAN').mean()
customers_labels_DBSCAN

Which algorithm appears to perform better?

In [ ]:
# Your observations here
ax = sns.heatmap(customers_labels_DBSCAN.corr(), linewidths=.5)

# Bonus Challenge 2 - Changing K-Means Number of Clusters

As we mentioned earlier, we don't need to worry about the number of clusters with DBSCAN because it automatically decides that based on the parameters we send to it. But with K-Means, we have to supply the `n_clusters` param (if you don't supply `n_clusters`, the algorithm will use `8` by default). You need to know that the optimal number of clusters differs case by case based on the dataset. K-Means can perform badly if the wrong number of clusters is used.

In advanced machine learning, data scientists try different numbers of clusters and evaluate the results with statistical measures (read [here](https://en.wikipedia.org/wiki/Cluster_analysis#External_evaluation)). We are not using statistical measures today but we'll use our eyes instead. In the cells below, experiment with different number of clusters and visualize with scatter plots. What number of clusters seems to work best for K-Means?

In [ ]:
# Your code here
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

def get_quick_cluster_visualization(clusters):
    customers_scale1, _ = make_blobs(n_samples=440, centers=clusters, cluster_std=1.5)

    kmeans = KMeans(n_clusters=clusters)
    kmeans.fit(customers_scale1)
    y_pred = kmeans.predict(customers_scale1)

    plt.figure(figsize=(6,6))
    plt.scatter(customers_scale1[:, 0], customers_scale1[:, 1], c=y_pred)

    centers = kmeans.cluster_centers_
    plt.scatter(centers[:, 0], centers[:, 1], c='red', s=50, alpha=0.50)
    plt.title('{} clusters visualization'.format(clusters))

    plt.show()
    
get_quick_cluster_visualization(4)

In [ ]:
ax0 = get_quick_cluster_visualization(1)
ax1 = get_quick_cluster_visualization(2)
ax2 = get_quick_cluster_visualization(3)
ax3 = get_quick_cluster_visualization(5)
ax4 = get_quick_cluster_visualization(7)
ax5 = get_quick_cluster_visualization(11)

# Bonus Challenge 3 - Changing DBSCAN `eps` and `min_samples`

Experiment changing the `eps` and `min_samples` params for DBSCAN. See how the results differ with scatter plot visualization.

In [ ]:
# Your code here


In [ ]:
# Your comment here